In [ ]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
data = {file.stem: pd.read_csv(file) for file in Path("data").glob("*.csv")}
data.keys()

In [ ]:
explore = data["train"]

In [ ]:
def data_prep(df):

In [ ]:
def sum_survived(df):
    sur = df["Survived"].sum()
    return pd.Series({"Survived_1": sur, "Survived_0": df.shape[0] - sur})
explore.groupby("Sex").apply(sum_survived).melt()

In [ ]:
explore

In [ ]:
def sum_survived(df):
    sur = df["Survived"].sum()
    return pd.Series({"Survived": sur, "Died": df.shape[0] - sur})

for column in explore.columns:
    if column in ["Survived","PassengerId", "Name"]:
        continue
    grouped = explore.groupby(column).apply(sum_survived).reset_index().melt(id_vars=column)
    sns.barplot(data=grouped, x=column, y="value", hue="variable")
    plt.show()

In [ ]:
# Fill nan age values
data["train"].info()

In [ ]:
def set_nan_age(df):
    med = df["Age"].median()
    df["Age"] = df["Age"].fillna(med)
    return df
train_data = data["train"].groupby(["Pclass", "Sex", "Embarked"]).apply(set_nan_age)
test_data = data["test"].groupby(["Pclass", "Sex", "Embarked"]).apply(set_nan_age)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Age"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=2)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
(y.shape[0] - np.sum(model.predict(X) != y))/y.shape[0]

In [ ]:
data["test"]["Survived"] = predictions

In [ ]:
data["test"][["PassengerId", "Survived"]].to_csv("submission.csv", index=False)

In [ ]:
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

dataset = tf.data.Dataset.from_tensor_slices((X.values.astype(float), y.values.astype(int)))
train_dataset = dataset.shuffle(X.shape[0]).batch(1)

# X_train, X_test, y_train, y_test = train_test_split(X, Y)

def create_baseline():

    model = tf.keras.Sequential([
      tf.keras.layers.Dense(16, input_shape=(X.shape[1],)),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )
    return model

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=200, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# model.fit(train_dataset, epochs=100)

In [ ]:
predictions = model(X.values.astype(np.float)).numpy()

(y.shape[0] - np.sum(np.squeeze(np.round(predictions)) != y))/y.shape[0]